In [2]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import KNeighborsClassifier #Supervised
from sklearn.neural_network import MLPClassifier
from sklearn import svm # Supervised 
from sklearn.tree import DecisionTreeClassifier

from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from numpy import load, save
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from collections import Counter
import time
from sklearn.metrics import f1_score
from tqdm import tqdm

import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import numpy.linalg as npl
#from decomp_utils import *
from numpy import load,save 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.neighbors import KernelDensity
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore")

In [3]:
plt.rcParams["figure.figsize"] = (20,10)
import sys 
sys.path.append(r'C:\Users\nmadhar\Desktop\Conv_deep\ToSubmit\Code')
from pca_ad_utils import *
from decomp_utils import *
from customizedLossSynthetic import *
from LocationAnomalyPCA import *

# I. Import Data 

In [4]:
X_traindf = pd.read_csv(r'C:\Users\nmadhar\Desktop\Conv_deep\ToSubmit\Data\XTrainLoc.csv').drop(['Unnamed: 0'],axis=1)
y_traindf = pd.read_csv(r'C:\Users\nmadhar\Desktop\Conv_deep\ToSubmit\Data\yTrainLoc.csv').drop(['Unnamed: 0'],axis=1)

X_testdf = pd.read_csv(r'C:\Users\nmadhar\Desktop\Conv_deep\ToSubmit\Data\XTestLoc.csv').drop(['Unnamed: 0'],axis=1)
y_testdf = pd.read_csv(r'C:\Users\nmadhar\Desktop\Conv_deep\ToSubmit\Data\yTestLoc.csv').drop(['Unnamed: 0'],axis=1)

Xtrain = np.array(X_traindf.iloc[:,:-3])
Xtest = np.array(X_testdf.iloc[:,:-3])
ytrain = np.array(y_traindf.iloc[:,:-3])
ytest = np.array(y_testdf.iloc[:,:-3])


ytrain_loc = [list(y_traindf.iloc[i,:]).index(1)for i in range(y_traindf.shape[0]) ]

ytest_loc = [list(y_testdf.iloc[i,:]).index(1) for i in range(y_testdf.shape[0]) ]
ntrain,d = Xtrain.shape
ntest = Xtest.shape[0]


In [5]:
n = int(Xtrain.shape[0]*206)
xtrain_flat = Xtrain.reshape(Xtrain.shape[0]*Xtrain.shape[1],1)[:n,0].reshape(-1,1)
ytrain_flat = ytrain.reshape(ytrain.shape[0]*ytrain.shape[1],1)[:n,0].reshape(-1,1)
xtest_flat = Xtest.reshape(Xtest.shape[0]*Xtest.shape[1],1)[:].reshape(-1,1)
ytest_flat = ytest.reshape(ytest.shape[0]*ytest.shape[1],1)[:].reshape(-1,1)


In [6]:
ScoresDFTrain = pd.DataFrame(columns=['Algo','Accuracy','Recall','Precision','F1'])
ScoresDFTrain['Algo'] = ['IF','LOF','KNN','DBSCAN','SVM','PCA NN AD']
ScoresDFTest = pd.DataFrame(columns=['Algo','Accuracy','Recall','Precision','F1'])
ScoresDFTest['Algo'] = ['IF','LOF','KNN','DBSCAN','SVM','PCA NN AD']

## II. Testing algorithms for outlier detection 

### 1. Isolation forest 

In [7]:

clf_if = IsolationForest(random_state=0).fit(xtrain_flat)
ypredTrainIF = redefine_if_lof_pred(clf_if.predict(xtrain_flat))
s = time.time()
ypredTestIF = redefine_if_lof_pred(clf_if.predict(xtest_flat))
timeIF = time.time() - s

print(f'Exe. time :{timeIF}\n')

print('-------------------Train score-------------------')
print_scores(ytrain_flat,ypredTrainIF)

print('-------------------Test score-------------------')
print_scores(ytest_flat,ypredTestIF)

Exe. time :14.504970073699951

-------------------Train score-------------------
Accuracy on Train : 0.8979094736427932
Recall on Train : 0.2470871327254306
Precision on Train : 0.012038453944120842
f1-score on Train : 0.02295834313956225

-------------------Test score-------------------
Accuracy on Train : 0.7093761214664024
Recall on Train : 0.49020094269412057
Precision on Train : 0.008190639624291713
f1-score on Train : 0.016112066927047235



In [10]:
ScoresLocIFTrain = evalalg(ytrain_flat,ypredTrainIF)
ScoresLocIFTest = evalalg(ytest_flat,ypredTestIF)

print(ScoresLocIFTrain)
print(ScoresLocIFTest)

(0.8979094736427932, 0.2470871327254306, 0.012038453944120842, 0.02295834313956225)
(0.7093761214664024, 0.49020094269412057, 0.008190639624291713, 0.016112066927047235)


In [8]:
print(pd.DataFrame(ypredTrainIF,columns=['target'])['target'].value_counts())
print(pd.DataFrame(ypredTestIF,columns=['target'])['target'].value_counts())

0    1464512
1     162064
Name: target, dtype: int64
0    589135
1    241251
Name: target, dtype: int64


In [11]:
ScoresDFTrain.iloc[0,1:] =  ScoresLocIFTrain
ScoresDFTest.iloc[0,1:] =  ScoresLocIFTest


### 2. Local Outlier factor

In [12]:

clf_lof = LocalOutlierFactor(n_neighbors=2,novelty= True)
clf_lof.fit(xtrain_flat)


ypredTrainLOF = redefine_if_lof_pred(clf_lof.predict(xtrain_flat))
s = time.time()
ypredTestLOF = redefine_if_lof_pred(clf_lof.predict(xtest_flat))
timeLOF = time.time()-s
print(f'Exe. time :{timeLOF}\n')
print('-------------------Train score-------------------')
print_scores(ytrain_flat,ypredTrainLOF)

print('-------------------Test score-------------------')
print_scores(ytest_flat,ypredTestLOF)

Exe. time :2.2869629859924316

-------------------Train score-------------------
Accuracy on Train : 0.9951056698242197
Recall on Train : 0.0
Precision on Train : 0.0
f1-score on Train : 0.0

-------------------Test score-------------------
Accuracy on Train : 0.020659066988123597
Recall on Train : 1.0
Precision on Train : 0.004932322804681975
f1-score on Train : 0.009816228800196763



In [13]:
ScoresLocLOFTrain = evalalg(ytrain_flat,ypredTrainLOF)
ScoresLocLOFTest = evalalg(ytest_flat,ypredTestLOF)
print(ScoresLocLOFTrain)
print(ScoresLocLOFTest)

(0.9951056698242197, 0.0, 0.0, 0.0)
(0.020659066988123597, 1.0, 0.004932322804681975, 0.009816228800196763)


In [14]:
print(pd.DataFrame(ypredTrainLOF,columns=['target'])['target'].value_counts())
print(pd.DataFrame(ypredTestLOF,columns=['target'])['target'].value_counts())


0    1626511
1         65
Name: target, dtype: int64
1    817262
0     13124
Name: target, dtype: int64


In [15]:
ScoresDFTrain.iloc[1,1:] =  ScoresLocLOFTrain
ScoresDFTest.iloc[1,1:] =  ScoresLocLOFTest

### 3. K nearest neighbors 

In [16]:
clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn.fit(xtrain_flat,ytrain_flat)
ypredTrainKNN = clf_knn.predict(xtrain_flat)
s = time.time()
ypredTestKNN = clf_knn.predict(xtest_flat)
timeKNN = time.time()-s
print(f'Exe. time :{timeKNN}\n')
print('-------------------Train score-------------------')
print_scores(ytrain_flat,ypredTrainKNN)

print('-------------------Test score-------------------')
print_scores(ytest_flat,ypredTestKNN)

Exe. time :14.188249588012695

-------------------Train score-------------------
Accuracy on Train : 0.9999993852116348
Recall on Train : 1.0
Precision on Train : 0.9998733696340383
f1-score on Train : 0.999936680807953

-------------------Test score-------------------
Accuracy on Train : 0.9555845112995643
Recall on Train : 0.13148102207888862
Precision on Train : 0.015629146884491758
f1-score on Train : 0.02793737810342101



In [17]:
ScoresLocKNNTrain = evalalg(ytrain_flat,ypredTrainKNN)
ScoresLocKNNTest = evalalg(ytest_flat,ypredTestKNN)

print(ScoresLocKNNTrain)
print(ScoresLocKNNTest)

(0.9999993852116348, 1.0, 0.9998733696340383, 0.999936680807953)
(0.9555845112995643, 0.13148102207888862, 0.015629146884491758, 0.02793737810342101)


In [18]:
print(pd.DataFrame(ytrain_flat,columns=['target'])['target'].value_counts())
print(pd.DataFrame(ytest_flat,columns=['target'])['target'].value_counts())

0.0    1618680
1.0       7896
Name: target, dtype: int64
0.0    826355
1.0      4031
Name: target, dtype: int64


In [19]:
ScoresDFTrain.iloc[2,1:] =  ScoresLocKNNTrain
ScoresDFTest.iloc[2,1:] =  ScoresLocKNNTest

### 4. DBSCAN 

In [22]:
clf_dbscan_train = DBSCAN(eps = .2, 
 metric='euclidean', 
 min_samples = 2,
 n_jobs = -1).fit_predict(xtrain_flat)

ypredTrainDBSCAN = list(clf_dbscan_train)
ypredTrainDBSCAN  =  redefine_pred_DBSCAN(ypredTrainDBSCAN)
print(f'Accuracy on Train : {accuracy_score(ytrain_flat,ypredTrainDBSCAN)}')
s = time.time()
clf_dbscan_test = DBSCAN(eps = .2, 
 metric='euclidean', 
 min_samples = 2,
 n_jobs = -1).fit_predict(xtest_flat)

ypredTestDBSCAN = list(clf_dbscan_test)
timeDBSCAN = time.time()
print(f'Exe. time :{timeDBSCAN}\n')

ypredTestDBSCAN  =  redefine_pred_DBSCAN(ypredTestDBSCAN)
ypredTestDBSCAN = list(clf_dbscan_test)
print(f'Accuracy on Test : {accuracy_score(ytest_flat,ypredTestDBSCAN)}')

MemoryError: 

### 5. SVM 

In [ ]:
clf_svm = svm.SVC()

clf_svm.fit(xtrain_flat,ytrain_flat)
ypredTrainSVM = clf_svm.predict(xtrain_flat)
s = time.time()
ypredTestSVM = clf_svm.predict(xtest_flat)
timeSVM = time.time()-s

print(f'Exe. time :{timeSVM}')

print(f'Accuracy on Train : {accuracy_score(ytrain_flat,ypredTrainSVM)}' )
print(f'Accuracy Train Zero model : {accuracy_score(ytrain_flat,dummy_train)}')

print(f'Accuracy on Test : {accuracy_score(ytest_flat,ypredTestSVM)}' )
print(f'Accuracy Test Zero model : {accuracy_score(ytest_flat,dummy_test)}')

#clf_svm = svm.SVC()
#clf_svm.fit(xtest,ytest)
#ypredTrainSVM = clf_svm.predict(xtrain)
#ypredTestSVM = clf_svm.predict(xtest)
#print(f'Accuracy on Train (fit on test) : {accuracy_score(ytrain,ypredTrainSVM)}' )
#print(f'Accuracy on Test (fit on test): {accuracy_score(ytest,ypredTestSVM)}' )


C:\Users\nmadhar\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


### 6. PCA NN

In [20]:
# with PCA approach
k = 20
norms_train, norms_test =  modelPCALocation(Xtrain,Xtest,k)

pred_loc_train = dummyprediction(norms_train)
scoresTrain = EvaluationPred(yPred = pred_loc_train,yTrue=ytrain_loc)
s = time.time()
pred_loc_test = dummyprediction(norms_test)
timePCANN = time.time()-s
print(f'Exe. time :{timePCANN}')

scoresTest = EvaluationPred(yPred = pred_loc_test,yTrue=ytest_loc)
ScoresPCANNTrain = [scoresTrain[0],scoresTrain[2],scoresTrain[1],scoresTrain[-1]]

ScoresPCANNTest = [scoresTest[0],scoresTest[2],scoresTest[1],scoresTest[-1]]


printRes(ScoresPCANNTrain,'Train')
printRes(ScoresPCANNTest,'Test')

100%|███████████████████████████████████████████████████████████████████████████| 4031/4031 [00:00<00:00, 12004.04it/s]


Exe. time :0.0020041465759277344
------------Scores on Train-------------
Accuracy: 0.8965298885511651
Recall: 0.8981681124332016
Precision: 0.8965298885511651
F1-score: 0.896823274794882

------------Scores on Test-------------
Accuracy: 0.9439345075663607
Recall: 0.9448988449153181
Precision: 0.9439345075663607
F1-score: 0.943797990883878



0

In [21]:
ScoresDFTrain.iloc[5,1:] =  ScoresPCANNTrain
ScoresDFTest.iloc[5,1:] =  ScoresPCANNTest

In [22]:
ScoresDFTrain

,Algo,Accuracy,Recall,Precision,F1
0,IF,0.897909,0.247087,0.012038,0.022958
1,LOF,0.995106,0.0,0.0,0.0
2,KNN,0.999999,1.0,0.999873,0.999937
3,DBSCAN,NaN,NaN,NaN,NaN
4,SVM,NaN,NaN,NaN,NaN
5,PCA NN AD,0.89653,0.898168,0.89653,0.896823


In [23]:
ScoresDFTest

,Algo,Accuracy,Recall,Precision,F1
0,IF,0.709376,0.490201,0.008191,0.016112
1,LOF,0.020659,1.0,0.004932,0.009816
2,KNN,0.955585,0.131481,0.015629,0.027937
3,DBSCAN,NaN,NaN,NaN,NaN
4,SVM,NaN,NaN,NaN,NaN
5,PCA NN AD,0.943935,0.944899,0.943935,0.943798


In [24]:
timeIF,timeLOF,timeKNN,timePCANN

(14.504970073699951,
 2.2869629859924316,
 14.188249588012695,
 0.0020041465759277344)